# Asistencia

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime as dt

from modules.tools import changeDF

In [ ]:
notas_parciales = True
carga = True
final = True

# Clean

In [ ]:
dfDict = {
    'SEMANA': '',
    'CORREO': 'Dirección de correo electrónico',
    'GENERO': 'Género',
    'GRUPO': 'Grupo al que asistió',
    'CALIFICACION': 'Califique de 1 a 5 el entrenamiento',
    'CARRERA': ['Programa académico', 'Programa académico.1', 'Programa académico.2', 'Programa académico.3', 'Programa académico.4'],
    'COMENTARIOS': 'Deja algún comentario a tu entrenador (Opcional)'
}

In [ ]:
data = pd.read_csv('./DATA/Asistencia.csv')

dictDF = changeDF(data, dfDict)

In [ ]:
#first = dt.datetime(2023, 8, 8)
first = dt.datetime(2023, 2, 27)

for d in list(data['Marca temporal']):
    week = dt.datetime.strptime(d, '%d/%m/%Y %H:%M:%S') - first
    dictDF['SEMANA'].append(week.days//7)

title = 'Resumen de la semana ' + str(dictDF['SEMANA'][-1])

In [ ]:
df = pd.DataFrame(dictDF)

df.head()

## Notas de los parciales

In [ ]:
order = df.groupby(by=['CORREO'], as_index=False).count().sort_values(by='SEMANA', ascending=False)

lista = list(order[order['SEMANA'] >= 5]['CORREO'])

In [ ]:
notDict = {
    'CORREO': list(),
    'PARCIAL 1': list(),
    'PARCIAL 2': list(),
    'PARCIAL 3': list(),
    'PARCIAL 4': list(),
    'FINAL': list()
}

equal = {
    'PARCIAL 1': 'P1',
    'PARCIAL 2': 'P2',
    'PARCIAL 3': 'P3',
    'PARCIAL 4': 'P4',
    'FINAL': 'Final'
}

notas = pd.read_csv('./DATA/Notas_Entrenados.csv')

for e in lista:
    notDict['CORREO'].append(e)
    for k,v in equal.items():
        try:
            notDict[k].append(float(notas[notas['E-MAIL'] == e][v]))
        except TypeError:
            notDict[k].append(0)

dfNotas = pd.DataFrame(notDict)

dfNotas

## Carga academica

In [ ]:
resume = {
    'EMAIL': list(),
    'SESIONES': list(),
    'CREDITOS': list(),
    'CARRERA': list(),
    'ENTRENAMIENTOS': list()
}

title = 'Resumen final del semestre 2023-2S'

In [ ]:
sia = pd.read_csv('./DATA/2023_1S.csv')

email = set(df['CORREO'])

for e in email:
    resume['EMAIL'].append(e)
    resume['SESIONES'].append(df[df['CORREO'] == e]['SEMANA'].count())
    resume['CREDITOS'].append(sia[sia['EMAIL'] == e]['CREDITOS'].sum())
    c = list(df[df['CORREO'] == e]['CARRERA'])
    resume['CARRERA'].append(c[0])
    resume['ENTRENAMIENTOS'].append(df[df['CORREO'] == e]['CALIFICACION'].mean())

dk = pd.DataFrame(resume)
dk.head(5)

# Analisis y Estadisticas

In [ ]:
plt.style.use('ggplot')

## Resumen del semestre

In [ ]:
weekly = df.groupby(['SEMANA', 'GRUPO', 'GENERO']).count()

In [ ]:
fig = plt.figure(figsize=(10, 5))

sns.lineplot(
    data=weekly,
    x='SEMANA',
    y='CORREO',
    hue='GRUPO',
    marker='o',
    errorbar=('ci', 0),
    palette=sns.color_palette('tab10', 19)
)

plt.legend(loc=(1.05,0))
plt.ylabel('ASISTENCIA')
plt.savefig('images/SemanaAsistenciaGrupo.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))

sns.histplot(
    data=df,
    x='SEMANA',
    hue='GENERO',
    multiple='dodge'
)

plt.ylabel('ASISTENCIA')
#plt.title('Asistencia por genero a lo largo del semestre')
plt.savefig('images/SemanaAsistenciaGenero.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

In [ ]:
sns.histplot(
    data=df,
    y='CARRERA',
    hue='GENERO',
    multiple='dodge'
)

plt.xlabel('ASISTENCIA')
#plt.title('Asistencia por carrera filtrado por genero')
plt.savefig('images/AsistenciaCarreraGenero.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

## Analisis ultima semana

In [ ]:
lastWeek = df[df['SEMANA'] == list(df['SEMANA'])[-1]]

lastWeek.head()

In [ ]:
fig = plt.figure(figsize=(8, 5))

sns.histplot(
    data=lastWeek,
    y='GRUPO'
)

plt.xlabel('ASISTENCIA')
#plt.title('Asistencia por grupos en la ultima semana')
plt.savefig('images/AsistenciaGrupo.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))

sns.histplot(
    data=lastWeek,
    y='GRUPO',
    hue='GENERO',
    multiple='dodge'
)

plt.xlabel('ASISTENCIA')
#plt.title('Asistencia por grupo en la ultima semana, por genero')
plt.savefig('images/GrupoAsistenciaGenero.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

## Analisis de las notas de los parciales

In [ ]:
graph = {
    'PARCIAL 1': 0,
    'PARCIAL 2': 1,
    'PARCIAL 3': 2,
    'PARCIAL 4': 3
}

fig, ax = plt.subplots(4, 1, figsize=[10, 20])

for k,v in graph.items():
    sns.histplot(
        data=dfNotas,
        x=k,
        ax=ax[v]
    )
    ax[v].set_ylabel('ESTUDIANTES')

plt.savefig('images/NotaDefinitiva.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

## Resumen final del semestre

In [ ]:
fig = plt.figure(figsize=(8, 5))

sns.histplot(
    data=dk,
    x='SESIONES'
)

plt.ylabel('ESTUDIANTES')
plt.xlim([0, 61])
#plt.title('Numero de estudiantes por sesiones asistidas')
plt.savefig('images/SesionesEstudiantes.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

In [ ]:
sns.histplot(
    data=dk,
    x='CREDITOS'
)

plt.ylabel('ESTUDIANTES')
#plt.title('Carga academica de los estudiantes')
plt.savefig('images/EstudiantesCreditos.jpg', format='jpg', bbox_inches='tight', dpi='figure')
plt.show()

# Informe

In [ ]:
dl = pd.read_excel('MinasContigo.xlsx', sheet_name='Entrenadores')

entrenadores = '<table>'

x = dl['GRUPOS'].count()

for y in range(0, x, 1):
    entrenadores += '<tr><td>' + dl['GRUPOS'][y] + '</td><td>' + dl['NOMBRE'][y] + '</td><td>' + dl['EMAIL'][y] + '</td></tr>'

entrenadores += '</table>'

In [ ]:
listComments = str()

notcomment = ['.']

for g in list(dl['GRUPOS']):
    listComments += '\n### ' + g + '\n'
    zero = True
    for c in lastWeek[lastWeek['GRUPO'] == g]['COMENTARIOS']:
        try:
            if notcomment.count(c) == 0:
                listComments += '\n* ' + c + '\n'
                zero = False
        except TypeError:
            pass
    if zero:
        listComments += '\n* Sin Comentarios\n'

In [ ]:
with open('README.md', 'w') as file:
    file.write('# Entrenamientos MinasContigo')
    file.write('\n## ' + title + '\n')
    file.write('\n' + entrenadores + '\n')
    file.write('\n### Asistencia en cada grupo durante la ultima semana\n')
    file.write('\n![](images/AsistenciaGrupo.jpg)\n')
    file.write('\n#### Asistencia en cada grupo durante la ultima semana filtrado por genero\n')
    file.write('\n![](images/GrupoAsistenciaGenero.jpg)\n')
    file.write('\n### Historico de asistencias por grupo\n')
    file.write('\n![](images/SemanaAsistenciaGrupo.jpg)\n')
    file.write('\n#### Historico de asistencias por grupo, filtrado por genero\n')
    file.write('\n![](images/SemanaAsistenciaGenero.jpg)\n')
    file.write('\n### Distribucion de las carreras, filtrada por genero\n')
    file.write('\n![](images/AsistenciaCarreraGenero.jpg)\n')
    if final:
        file.write('\n### Numero de sesiones por estudiantes que han asistido\n')
        file.write('\n![](images/SesionesEstudiantes.jpg)\n')
    if carga:
        file.write('\n### Carga academica de los entrenados\n')
        file.write('\n![](images/EstudiantesCreditos.jpg)\n')
    if notas_parciales:
        file.write('\n### Notas de los estudiantes, en los parciales\n')
        file.write('\n![](images/NotaDefinitiva.jpg)\n')
    file.write('\n## Comentarios en la ultima semana\n')
    file.write(listComments)
    file.close()